In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 33.8 MB/s eta 0:00:00


In [12]:
import sys
import os
import warnings
import json
import re
import numpy as np
import pandas as pd
import torch
from copy import deepcopy
from enum import Enum
from typing import Dict, List, Tuple, Any
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from sklearn.preprocessing import StandardScaler

# --- DEPENDENCY CHECKS ---
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ 'openai' library not found. LLM Assistant will be DISABLED.")

try:
    import tiktoken
except ImportError:
    tiktoken = None
    print("⚠️ Note: 'tiktoken' not found. Using simple character count for token limits.")

# BoTorch imports
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qLogNoisyExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.utils.transforms import normalize, unnormalize, standardize

warnings.filterwarnings('ignore')

# -----------------------------------------------------------------------------
# 0. CONFIGURATION & PROMPTS
# -----------------------------------------------------------------------------

OPENAI_API_KEY = ""

PROMPTS = {
    "experiment_overview": """
You are an expert scientist assisting with a chemical optimization experiment.
Goal: Maximize [target].
Description: [description]
Parameters:
[parameters_and_bounds]
Constraint: [constraint]
Domain: [domain]

Please provide a brief overview of the experiment and the challenges involved.
""",
    "starter": """
Based on the experiment description, generate [n_hypotheses] initial hypotheses for points that might maximize the target.
Consider the chemical properties and potential interactions.
Return the response in JSON format with fields: "comment" (string) and "hypotheses" (list of dicts with "name", "rationale", "confidence", "points").
Each "point" must be a dictionary of parameter names and values.
""",
    "comment_selection": """
Current Iteration: [iteration]
We have trained a Gaussian Process model and generated candidate points using qLogNEI (Log Noisy Expected Improvement).

Top Suggestions:
[suggestions]

Historical Data (Top/Recent):
[dataset]

Please analyze these suggestions. Which ones seem most chemically promising based on the historical data and your scientific knowledge?
Select the best ones or suggest modifications if they violate "chemical sense" (though strictly respect the bounds).
Return JSON with "comment" and "hypotheses" (where "points" are the selected/refined candidates).
""",
    "conclusion": """
The optimization batch is complete.
Data Summary:
[dataset]

Provide a final conclusion on the findings and recommended next steps.
"""
}

# -----------------------------------------------------------------------------
# 1. MOCK BORA CLASSES (To satisfy Assistant dependencies)
# -----------------------------------------------------------------------------

class MockParameter:
    def __init__(self, name, bounds, step=None):
        self.name = name
        self.bounds = bounds
        self.description = "Chemical parameter"
        self.type = "continuous"
        self.step = step
        self.categories = []

    def get_bounds(self):
        return self.bounds

    def is_valid_value(self, v):
        return self.bounds[0] <= v <= self.bounds[1]

class MockTarget:
    def __init__(self, name):
        self.name = name

class MockConstraint:
    def __init__(self, description):
        self.description = description
        self.constraint = None 
    def eval(self, **kwargs): return 0
    def allowed(self, val): return [True]

class MockExperiment:
    def __init__(self, name, parameters, target_name):
        self.name = name
        self.parameters = parameters
        self.dim = len(parameters)
        self.type = "continuous"
        self.constraint = MockConstraint("Sum of components (excluding P10) <= 5")
        self.domain = "Chemical Formulation"
        self.target = MockTarget(target_name)
        self.default_precision = 4
        self.keys = [p.name for p in parameters]
        self.pbounds = {p.name: p.bounds for p in parameters}
        self.description = "Optimize a chemical formula to maximize the target yield."
    
    def get_parameter(self, name):
        for p in self.parameters:
            if p.name == name: return p
        return None

# -----------------------------------------------------------------------------
# 2. ASSISTANT CLASS (Adapted)
# -----------------------------------------------------------------------------

class CommentType(Enum):
    PREOPTIMIZATION = 0
    POINTS = 1
    CONCLUSION = 3

class BaseComment:
    def __init__(self, response: str, iteration: int) -> None:
        self._iteration = iteration
        self._comment = response
        self.type = None
        self.hypotheses = []

    def __str__(self): return str(self._comment)
    @property
    def comment(self): return self._comment
    @property
    def iteration(self): return self._iteration
    @property
    def is_valid(self): return self._comment is not None

class Comment(BaseComment):
    def __init__(self, response: str, iteration: int, experiment, llm_model, api_key):
        super().__init__(response, iteration)
        self._experiment = experiment
        self._comment = ""
        self._hypotheses = []
        
        try:
            # Parse JSON
            json_match = re.search(r"\{.*\}", response, re.DOTALL)
            if json_match:
                data = json.loads(json_match.group())
                self._comment = data.get("comment", "")
                self._hypotheses = data.get("hypotheses", [])
        except:
            self._comment = response # Fallback if not valid JSON

    def __str__(self):
        return json.dumps({"comment": self._comment, "hypotheses": self._hypotheses}, indent=2)

class Assistant:
    def __init__(self, api_key, experiment, log_path, llm_model="gpt-4o-mini"):
        self._api_key = api_key
        self._experiment = experiment
        self._log_path = log_path
        self._model = llm_model
        
        if OPENAI_AVAILABLE:
            self._client = OpenAI(api_key=api_key)
        else:
            self._client = None
            
        self._comments = []
        self._chat_history = [
            {"role": "system", "content": "You are BORA, an AI scientist for Bayesian Optimization."}
        ]
        
        # Initialize encoder if tiktoken is available
        self._encoding = None
        if tiktoken:
            try:
                self._encoding = tiktoken.encoding_for_model(llm_model)
            except:
                pass

    def _get_token_count(self, text):
        if self._encoding:
            return len(self._encoding.encode(text))
        else:
            return len(text) // 4

    def _chat_completion(self, messages):
        if not self._client: return None
        try:
            completion = self._client.chat.completions.create(
                model=self._model, messages=messages, temperature=0.7
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(f"LLM Error: {e}")
            return None

    def _fill_prompt(self, template, extra_subs={}):
        txt = template
        txt = txt.replace("[target]", self._experiment.target.name)
        txt = txt.replace("[description]", self._experiment.description)
        txt = txt.replace("[constraint]", self._experiment.constraint.description)
        txt = txt.replace("[domain]", self._experiment.domain)
        
        # Parameters
        p_str = ""
        for p in self._experiment.parameters:
            p_str += f"- {p.name}: bounds={p.get_bounds()}\n"
        txt = txt.replace("[parameters_and_bounds]", p_str)
        
        for k, v in extra_subs.items():
            txt = txt.replace(f"[{k}]", str(v))
        return txt

    def pre_optimization_comment(self, n_hypotheses=3):
        if not OPENAI_AVAILABLE: return None
        
        prompt = self._fill_prompt(PROMPTS["starter"], {"n_hypotheses": n_hypotheses})
        self._chat_history.append({"role": "user", "content": prompt})
        
        resp = self._chat_completion(self._chat_history)
        if resp:
            comment = Comment(resp, 0, self._experiment, self._model, self._api_key)
            comment.type = CommentType.PREOPTIMIZATION
            self._comments.append(comment)
            self._chat_history.append({"role": "assistant", "content": resp})
            self._log_comment(comment)
            return comment
        return None

    def comment_and_select_point(self, data: pd.DataFrame, suggestions: pd.DataFrame):
        if not OPENAI_AVAILABLE: 
            print("⚠️ OpenAI library missing. Skipping LLM review.")
            return None
            
        data_summary = data.tail(10).to_string(index=False)
        sugg_str = suggestions.to_string(index=False)
        
        prompt = self._fill_prompt(PROMPTS["comment_selection"], {
            "iteration": len(data),
            "dataset": data_summary,
            "suggestions": sugg_str
        })
        
        # Check context window (simple check)
        if self._get_token_count(prompt) > 100000:
            print("⚠️ Prompt too long, truncating history...")
            data_summary = data.tail(5).to_string(index=False)
            prompt = self._fill_prompt(PROMPTS["comment_selection"], {
                "iteration": len(data),
                "dataset": data_summary,
                "suggestions": sugg_str
            })

        self._chat_history.append({"role": "user", "content": prompt})
        
        resp = self._chat_completion(self._chat_history)
        if resp:
            comment = Comment(resp, len(data), self._experiment, self._model, self._api_key)
            comment.type = CommentType.POINTS
            self._comments.append(comment)
            self._chat_history.append({"role": "assistant", "content": resp})
            self._log_comment(comment)
            return comment
        return None

    def _log_comment(self, comment):
        with open(self._log_path, "a", encoding="utf-8") as f:
            f.write(f"\n\n## Iteration {comment.iteration}\n{comment}\n")
            print(f"\n--- LLM Comment ---\n{comment.comment}\n-------------------")

# -----------------------------------------------------------------------------
# 3. BOTORCH OPTIMIZER (Gaussian Process + qLogNEI)
# -----------------------------------------------------------------------------

class BoTorchOptimizer:
    """BoTorch-based optimizer using Gaussian Process and qLogNEI"""
    def __init__(self, bounds: np.ndarray, random_state: int = 42):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dtype = torch.double
        self.bounds_np = bounds
        self.bounds_tensor = torch.tensor(bounds.T, device=self.device, dtype=self.dtype)
        torch.manual_seed(random_state)
        self.model = None
        self.train_y_mean = None
        self.train_y_std = None
        
    def fit(self, X: np.ndarray, y: np.ndarray):
        if np.isnan(X).any() or np.isnan(y).any():
            raise ValueError("Input X or y contains NaNs!")

        self.X_train = torch.tensor(X, device=self.device, dtype=self.dtype)
        self.y_train = torch.tensor(y.reshape(-1, 1), device=self.device, dtype=self.dtype)
        
        self.train_y_mean = self.y_train.mean()
        self.train_y_std = self.y_train.std()
        if self.train_y_std < 1e-9:
             self.train_y_std = torch.tensor(1.0, device=self.device, dtype=self.dtype)

        self.train_X_norm = normalize(self.X_train, self.bounds_tensor)
        self.train_y_std_norm = standardize(self.y_train)
        
        print(f"Fitting Gaussian Process model on {len(X)} samples...")
        self.model = SingleTaskGP(self.train_X_norm, self.train_y_std_norm)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_mll(mll)
        print(f"✓ GP model fitted.")

    def suggest_batch_qlognei(self, q: int = 10):
        """Use qLogNoisyExpectedImprovement with GP"""
        
        acq_func = qLogNoisyExpectedImprovement(
            model=self.model,
            X_baseline=self.train_X_norm,
        )
        
        # Constraints definition
        idx_p10 = 5
        idx_others = [i for i in range(self.bounds_tensor.shape[1]) if i != idx_p10]
        lower = self.bounds_tensor[0]
        upper = self.bounds_tensor[1]
        ranges = upper - lower
        sum_lower_others = lower[idx_others].sum()
        rhs_val = (sum_lower_others - 5.0).item()
        coeffs_tensor = -ranges[idx_others]
        indices_tensor = torch.tensor(idx_others, device=self.device, dtype=torch.long)
        
        inequality_constraints = [(indices_tensor, coeffs_tensor, rhs_val)]

        try:
            candidates_norm, _ = optimize_acqf(
                acq_function=acq_func, 
                bounds=torch.stack([torch.zeros_like(lower), torch.ones_like(upper)]),
                q=q, 
                num_restarts=10, 
                raw_samples=512, 
                inequality_constraints=inequality_constraints, 
                sequential=True
            )
        except Exception as e:
             print(f"Optimization warning: {e}. Falling back to unconstrained initialization.")
             candidates_norm, _ = optimize_acqf(
                acq_function=acq_func, 
                bounds=torch.stack([torch.zeros_like(lower), torch.ones_like(upper)]),
                q=q, 
                num_restarts=10, 
                raw_samples=512, 
                sequential=True
            )

        candidates_raw = unnormalize(candidates_norm, self.bounds_tensor).detach().cpu().numpy()
        
        # Post-process
        final_candidates = []
        for cand in candidates_raw:
            cand = self._apply_rounding_and_constraints(cand, idx_p10, idx_others)
            final_candidates.append(cand)
        return np.array(final_candidates)

    def _apply_rounding_and_constraints(self, candidate, idx_p10, idx_others):
        cand = candidate.copy()
        p10_val = round(cand[idx_p10] / 0.2) * 0.2
        cand[idx_p10] = max(1.2, p10_val)
        cand[idx_others] = np.round(cand[idx_others] / 0.25) * 0.25
        cand[idx_others] = np.maximum(0.0, cand[idx_others])
        while cand[idx_others].sum() > 5.0 + 1e-6:
            local_indices = np.argsort(cand[idx_others])[::-1]
            reduced = False
            for loc_i in local_indices:
                real_idx = idx_others[loc_i]
                if cand[real_idx] >= 0.25:
                    cand[real_idx] -= 0.25
                    reduced = True
                    break
            if not reduced: break
        return cand

    def predict(self, X: np.ndarray):
        """Predict using the GP Posterior"""
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, device=self.device, dtype=self.dtype)
            X_norm = normalize(X_tensor, self.bounds_tensor)
            
            posterior = self.model.posterior(X_norm)
            mu = posterior.mean * self.train_y_std + self.train_y_mean
            sigma = posterior.variance.sqrt() * self.train_y_std
            
            return mu.cpu().numpy().ravel(), sigma.cpu().numpy().ravel()

# -----------------------------------------------------------------------------
# 4. VISUALIZATION
# -----------------------------------------------------------------------------

def create_enhanced_visualization(suggestions, X, y, feature_cols, output_dir, model_wrapper):
    best_sugg = max(suggestions, key=lambda x: x['predicted_target'])
    best_params = best_sugg['params']
    n_features = len(feature_cols)
    cols_per_row = 5
    rows_needed = (n_features + cols_per_row - 1) // cols_per_row
    
    fig = plt.figure(figsize=(20, 5 * rows_needed + 5))
    gs = gridspec.GridSpec(rows_needed + 1, cols_per_row, height_ratios=[1]*rows_needed + [0.8], hspace=0.4, wspace=0.3)
    
    for i, col_name in enumerate(feature_cols):
        row = i // cols_per_row
        col = i % cols_per_row
        ax = fig.add_subplot(gs[row, col])
        
        x_min, x_max = model_wrapper.bounds_np[i]
        if x_min == x_max: view_min, view_max = x_min - 0.1, x_max + 0.1
        else: view_min, view_max = x_min - (x_max - x_min)*0.05, x_max + (x_max - x_min)*0.05
        
        x_grid = np.linspace(view_min, view_max, 100)
        X_visualize = np.tile(best_params, (100, 1))
        X_visualize[:, i] = x_grid
        y_pred_grid, y_std_grid = model_wrapper.predict(X_visualize)
        
        lower_bound = np.maximum(y_pred_grid - 1.96 * y_std_grid, 0)
        upper_bound = np.maximum(y_pred_grid + 1.96 * y_std_grid, 0)
        
        ax.plot(x_grid, np.maximum(y_pred_grid, 0), color='#2c3e50', linewidth=2.5, label='GP Mean')
        ax.fill_between(x_grid, lower_bound, upper_bound, color='#3498db', alpha=0.15, label='95% CI')
        ax.scatter(X[:, i], y, color='gray', alpha=0.4, s=30)
        ax.scatter([best_params[i]], [best_sugg['predicted_target']], color='#e74c3c', s=150, marker='*', zorder=10, edgecolor='black')
        ax.set_xlabel(col_name, fontsize=10, fontweight='bold')
        if col == 0: ax.set_ylabel("Predicted Target", fontsize=10)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(loc='best', fontsize=7, framealpha=0.8)

    ax_bar = fig.add_subplot(gs[rows_needed, :])
    top_n = sorted(suggestions, key=lambda x: x['predicted_target'], reverse=True)[:10]
    labels = [f"Best Hist.\n({y.max():.3f})"] + [f"#{k+1}" for k in range(len(top_n))]
    values = [y.max()] + [s['predicted_target'] for s in top_n]
    bar_colors = ['#34495e'] + ['#e74c3c' for _ in top_n]
    
    bars = ax_bar.bar(range(len(labels)), values, capsize=5, color=bar_colors, alpha=0.85, edgecolor='black')
    ax_bar.set_xticks(range(len(labels)))
    ax_bar.set_xticklabels(labels, fontsize=10)
    ax_bar.set_ylabel('Target Value', fontsize=11, fontweight='bold')
    ax_bar.set_title('Top Suggestions (qLogNEI)', fontsize=13)
    
    legend_elements = [Patch(facecolor='#34495e', edgecolor='black', label='Best Historical'),
                       Patch(facecolor='#e74c3c', edgecolor='black', label='qLogNEI Suggestion')]
    ax_bar.legend(handles=legend_elements, loc='upper right')
    
    plot_path = os.path.join(output_dir, 'optimization_landscape.png')
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()

# -----------------------------------------------------------------------------
# 5. MAIN LOOP (With LLM Assistant)
# -----------------------------------------------------------------------------

def optimize_dataset(filepath: str, n_suggestions: int = 12, output_dir: str = None):
    if output_dir is None: output_dir = os.path.dirname(os.path.abspath(filepath))
    os.makedirs(output_dir, exist_ok=True)
    
    print("=" * 80)
    print("Loading data from:", filepath)
    
    try:
        df = pd.read_excel(filepath) if filepath.endswith('.xlsx') else pd.read_csv(filepath)
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        return None
    
    feature_cols = [
        'AcidRed871_0gL', 'L-Cysteine-50gL', 'MethyleneB_250mgL', 'NaCl-3M', 
        'NaOH-1M', 'P10-MIX1', 'PVP-1wt', 'RhodamineB1_0gL', 'SDS-1wt', 'Sodiumsilicate-1wt'
    ]
    target_col = 'Target'
    
    df_clean = df[feature_cols + [target_col]].copy()
    for col in df_clean.columns: df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    df_clean.dropna(inplace=True)
    
    X = df_clean[feature_cols].values
    y = df_clean[target_col].values
    
    # --- SETUP LLM ASSISTANT ---
    mock_params = []
    # Index 5 is P10-MIX1 (step 0.2), others step 0.25
    for i, name in enumerate(feature_cols):
        min_b = max(0, X[:, i].min() * 0.9)
        max_b = max(X[:, i].max() * 1.1, 1e-6)
        if i == 5: # P10
            min_b = 1.0001
            max_b = max(max_b, 2.0)
            step = 0.2
        else:
            step = 0.25
        mock_params.append(MockParameter(name, [min_b, max_b], step))
    
    mock_exp = MockExperiment("Chemical Opt", mock_params, target_col)
    log_file = os.path.join(output_dir, "assistant_log.md")
    
    print("\n🤖 Initializing LLM Assistant...")
    assistant = Assistant(OPENAI_API_KEY, mock_exp, log_file)
    assistant.pre_optimization_comment()
    
    # --- RUN OPTIMIZATION ---
    print("\nInitializing Gaussian Process model...")
    bounds = np.array([p.bounds for p in mock_params])
    botorch_opt = BoTorchOptimizer(bounds=bounds)
    botorch_opt.fit(X, y)
    
    print(f"\nGenerating {n_suggestions} qLogNEI suggestions...")
    candidates = botorch_opt.suggest_batch_qlognei(q=n_suggestions)
    
    all_suggestions = []
    for cand in candidates:
        mu, std = botorch_opt.predict(cand.reshape(1, -1))
        all_suggestions.append({
            'method': 'qLogNEI', 
            'params': cand, 
            'predicted_target': float(mu[0]), 
            'uncertainty': float(std[0])
        })

    suggestions_df = pd.DataFrame([
        {'Method': s['method'], 
         **{feature_cols[i]: s['params'][i] for i in range(len(feature_cols))},
         'Predicted Target': s['predicted_target'], 
         'Uncertainty': s['uncertainty']} 
        for s in all_suggestions
    ])
    
    out_file = os.path.join(output_dir, 'optimization_suggestions.xlsx')
    suggestions_df.to_excel(out_file, index=False)
    print(f"✓ Suggestions saved to: {out_file}")
    
    # --- LLM REVIEW ---
    print("\n🤖 LLM is reviewing suggestions...")
    assistant.comment_and_select_point(df_clean, suggestions_df)

    create_enhanced_visualization(all_suggestions, X, y, feature_cols, output_dir, botorch_opt)
    return suggestions_df

if __name__ == "__main__":
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    input_file = "Training data.xlsx"
    
    if os.path.exists(input_file):
        optimize_dataset(filepath=input_file, n_suggestions=12, output_dir=desktop_path)
    else:
        print(f"❌ Could not find '{input_file}'.")

⚠️ Note: 'tiktoken' not found. Using simple character count for token limits.
Loading data from: Training data.xlsx

🤖 Initializing LLM Assistant...
LLM Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************7gwA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}

Initializing Gaussian Process model...
Fitting Gaussian Process model on 3 samples...
✓ GP model fitted.

Generating 12 qLogNEI suggestions...
✓ Suggestions saved to: /Users/michaelhuang/Desktop/optimization_suggestions.xlsx

🤖 LLM is reviewing suggestions...
LLM Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-****************************************************************************************

In [9]:
if __name__ == "__main__":
    # Auto-detect Desktop path
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    input_file = "HER_virtual_data.xlsx"
    
    if os.path.exists(input_file):
        optimize_dataset(
            filepath=input_file, 
            n_suggestions=12,
            output_dir=desktop_path 
        )
    else:
        print(f"❌ Could not find '{input_file}'. Please check the filename.")

Loading data from: HER_virtual_data.xlsx
Data: 812 valid experiments | Best Target: 27.7041

Initializing qUCB model...
✓ BoTorch GP model fitted on 812 samples

GENERATING SUGGESTIONS (qUCB ONLY)
Constraints: Sum(others) <= 5 | P10 > 1 | Discrete Steps

✓ Suggestions saved to: /Users/michaelhuang/Desktop/optimization_suggestions.xlsx
✓ Visualization saved to: /Users/michaelhuang/Desktop/optimization_landscape.png
